# Select database (i.e. staging, production, local authenticated, local unauthenticated)

In [ ]:
import requests
from obi_auth import get_token
from entitysdk.client import Client
from entitysdk.common import ProjectContext
from enum import Enum
import obi_one as obi

class DatabaseEnvironment(Enum):
    STAGING_DATABASE = "staging"
    # PRODUCTION_DATABASE = "production"
    LOCAL_UNAUTHENTICATED = "local_unauthenticated"
    LOCAL_AUTHENTICATED = "local_authenticated"

obi_one_api_url = "http://127.0.0.1:8100"

database_environment = DatabaseEnvironment.STAGING_DATABASE
# database_environment = DatabaseEnvironment.PRODUCTION_DATABASE
# database_environment = DatabaseEnvironment.LOCAL_UNAUTHENTICATED
# database_environment = DatabaseEnvironment.LOCAL_AUTHENTICATED

if database_environment == DatabaseEnvironment.STAGING_DATABASE:
    """
    - Uncomment in obi-one/.env.run-local before "make run-local":
        export ENTITYCORE_URL=https://staging.openbraininstitute.org/api/entitycore
    """
    entitycore_api_url = "https://staging.openbraininstitute.org/api/entitycore"
    virtual_lab_id=obi.LAB_ID_STAGING_TEST
    project_id=obi.PROJECT_ID_STAGING_TEST

# elif database_environment == DatabaseEnvironment.PRODUCTION_DATABASE:
#     """
#     For future: EntityCore not currently in production.
#     - Uncomment in obi-one/.env.run-local before "make run-local":
#         export ENTITYCORE_URL=https://www.openbraininstitute.org/api/entitycore
#     """
#     entitycore_api_url = "https://www.openbraininstitute.org/api/entitycore"

elif DatabaseEnvironment.LOCAL_UNAUTHENTICATED:
    """
    Not yet tested.
    - Launch entitycore locally (make run-local)
    - Add a morphology with a project name (see entitysdk)
    """
    entitycore_api_url = "https://127.0.1:8000/api/entitycore"

elif DatabaseEnvironment.LOCAL_AUTHENTICATED:
    """
    Not yet tested.
    - Change APP_DISABLE_AUTH to False in entitycore/.env.run-local
    - Launch entitycore locally (make run-local)
    - Add a morphology with a project name (see entitysdk morphology.ipynb example)
    """
    entitycore_api_url = "https://127.0.1:8000/api/entitycore"

else:
    raise ValueError(f"Unsupported environment: {database_environment}")


token = get_token(environment="staging")
project_context = ProjectContext(virtual_lab_id=virtual_lab_id, project_id=project_id)
client = Client(api_url=entitycore_api_url, project_context=project_context, token_manager=token)

# Fetch a morphology

In [19]:
from entitysdk.models.morphology import (
    ReconstructionMorphology,
)
kwargs = {"name": "ch150801A1"}
reconstruction_morphologies = client.search_entity(
    entity_type=ReconstructionMorphology, query=kwargs, limit=1
).one()

reconstruction_morphology_id = reconstruction_morphologies.id
print(reconstruction_morphology_id)

d5bad865-7e73-415c-9422-c62e2cec2bda


# Call the declared morphology metrics endpoint

In [20]:
# Construct the full endpoint URL
url = f"{obi_one_api_url}/declared/neuron-morphology-metrics/{reconstruction_morphology_id}"

# Set headers
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json"
}

# Optionally include optional headers if they are set
if virtual_lab_id:
    headers["virtual-lab-id"] = virtual_lab_id
if project_id:
    headers["project-id"] = project_id

# Make the GET request
response = requests.get(url, headers=headers)

# Check the response
if response.status_code == 200:
    data = response.json()
    print("Success:", data)
else:
    print(f"Error {response.status_code}: {response.text}")


Success: {'aspect_ratio': 0.7287205454901656, 'circularity': 0.7196932543874601, 'length_fraction_above_soma': 0.5171987414360046, 'max_radial_distance': 793.4895629882812, 'number_of_neurites': 7, 'soma_radius': 7.022326435526172, 'soma_surface_area': 619.6862957453552, 'total_length': 8737.680517092347, 'total_height': 1114.3104248046875, 'total_depth': 116.20946502685547, 'total_area': 11952.16464829219, 'total_volume': 1610.1819871486382, 'section_lengths': [76.3779296875, 24.82164764404297, 13.352164268493652, 37.46620559692383, 17.62110137939453, 124.3603286743164, 5.958928108215332, 50.47462463378906, 44.19041442871094, 17.077606201171875, 1.4204202890396118, 245.5657196044922, 21.29269027709961, 41.22689437866211, 20.829160690307617, 15.924124717712402, 182.13662719726562, 109.8696517944336, 26.792217254638672, 31.253459930419922, 18.245052337646484, 0.14462514221668243, 31.55167579650879, 14.158295631408691, 5.347115516662598, 176.0040740966797, 209.35443115234375, 186.6557922

# Call the generated morphology metrics endpoint

In [21]:
import requests

# Construct the full endpoint URL
url = f"{obi_one_api_url}/generated/morphology-metrics-run-grid"

# Prepare headers
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json",
    "Content-Type": "application/json"
}

# Optionally include optional headers if they are set
if virtual_lab_id:
    headers["virtual-lab-id"] = virtual_lab_id
if project_id:
    headers["project-id"] = project_id

# Construct request body — adjust this with the actual morphology metrics form data!
# Placeholder example:
request_body = {
  "type": "MorphologyMetricsForm",
  "initialize": {
    "type": "MorphologyMetricsForm.Initialize",
    "morphology": {
      "id_str": str(reconstruction_morphology_id),
    }
  }
}

# Make the POST request
response = requests.post(url, headers=headers, json=request_body)

# Check the response
if response.status_code == 200:
    data = response.json()
    print("Success:", data)
else:
    print(f"Error {response.status_code}: {response.text}")

Success: 


In [ ]:
from entitysdk.models.electrical_cell_recording import (
    ElectricalCellRecording,
)
kwargs = {"name": "S1FL_L5_DBC_cIR_4"}
Electrical_cell_recording = client.search_entity(
    entity_type=ElectricalCellRecording, query=kwargs, limit=1
).one()

Electrical_cell_recording_id = Electrical_cell_recording.id)
print(Electrical_cell_recording_id)

assets=[Asset(id=UUID('898ef0f4-d36f-4426-8af7-145cdb202004'), update_date=None, creation_date=None, path='S1FL_L5_DBC_cIR_4.nwb', full_path='public/a98b7abc-fc46-4700-9e3d-37137812c730/0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6/assets/electrical_cell_recording/98dece14-e154-4ff1-9c1d-34e1563cdd0a/S1FL_L5_DBC_cIR_4.nwb', is_directory=False, content_type='application/nwb', size=1657656, sha256_digest='96cc3960ce65133a79be19af9b8b5fa614eb42328cb2e4c67fbe0f96c51c28e2', status='created', meta={}, label='nwb')] id=UUID('98dece14-e154-4ff1-9c1d-34e1563cdd0a') update_date=datetime.datetime(2024, 1, 11, 7, 56, 56, 842835, tzinfo=TzInfo(UTC)) creation_date=datetime.datetime(2021, 6, 15, 9, 42, 25, 311000, tzinfo=TzInfo(UTC)) name='S1FL_L5_DBC_cIR_4' description='This dataset is about simulated electrophysiology traces for cell instance S1FL_L5_DBC_cIR_4. The dataset contains one distribution of the traces in NWB file format.' type='electrical_cell_recording' created_by=Person(id=UUID('4067c358-4ee2-4

# Call the declared electrophysiology recording metrics endpoint

In [ ]:
import requests

trace_id = str(Electrical_cell_recording_id)
url = f"{obi_one_api_url}/declared/electrophysiologyrecording-metrics/{trace_id}"

# Define query parameters
params = {
    "requested_metrics": ["AP_height", "spike_count"],
    "protocols": "step",
    "min_value": 0.1,
    "max_value": 0.4,
}

headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json",
}

if virtual_lab_id:
    headers["virtual-lab-id"] = virtual_lab_id
if project_id:
    headers["project-id"] = project_id

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()
    print("Success:", data)
else:
    print(f"Error {response.status_code}: {response.text}")

Success: {'feature_dict': {'step_0.25': {'AP_height': {'avg': -16.068817377571733, 'num_traces': 2, 'unit': 'mV'}, 'spike_count': {'avg': 2.0, 'num_traces': 2, 'unit': None}, 'stimulus_current': '0.32942111045122147 nA'}}}
